In [1]:
import torch 
import tiktoken
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
@dataclass
class GPTConfig:
    block_size:int = 1024 # this is max sequence len
    vocab_size:int = 50304 #50257 # total vocab including 256 bytes + 1 special token (<|endoftext|>) and 1000-257 BPE merges
    n_layer:int = 12 # number of layers 
    n_head:int = 12 # total number of attention heads
    n_embd: int = 768 # embedding dimension

In [3]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        n_head = config.n_head
        n_embd = config.n_embd
        
        assert n_embd % n_head == 0
        
        # query, key, value prjections all combined
        self.c_attn = nn.Linear(n_embd, 3 * n_embd)
        
        # output projection, after `v` is already multiplied with attention_scores
        self.c_proj = nn.Linear(n_embd, n_embd)
        
        self.c_proj.NANOGPT_SCALE_INIT=1

        block_size = config.block_size
        
        self.register_buffer('bias', torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size))
        
        self.n_embd = n_embd
        self.n_head = n_head

    def forward(self, x):
        B, T, C = x.size() # batch_size, sequence_len, embedding_dim (n_embd)
        # total dim = n_head * head_size
        # example GPT2 has 12 heads with each hs = 64 thus C= 12*64 = 768

        qkv = self.c_attn(x) # get combined qkv matix B, T, n_embd * 3(768*3=2304)

        q, k, v = qkv.split(self.n_embd, dim=2) # each item gets n_embd size, dimension against two 

        # b, seq, n_embd -> b, seq, n_heads, head_size -> b, n_heads, seq_len, head_size
        q = q.view(B, T, self.n_head, C//self.n_head).transpose(1, 2)
        # final-> bs, n_heads, seq_len, mini-n_head_embd

        k = k.view(B, T, self.n_head, C//self.n_head).transpose(1, 2)
        
        v = v.view(B, T, self.n_head, C//self.n_head).transpose(1, 2)
        
        # # print(f"shape of q: {q.shape}... shape of k : {k.shape}")
        
        # attn = (q @ k.transpose(-2, -1))/(math.sqrt(k.shape[-1]))

        # # apply masked fill at places where mask ==0, remember tril is lower triangle
        # attn = attn.masked_fill(mask = self.bias[ : , : , :T, :T] == 0, value=float('-inf'))
        
        # attn = F.softmax(attn, dim=-1)
        
        # y = attn @ v # B, n_heads, T/seq, T @ B, n_heads, T/Seq, head_size) -> B, n_heads, T, head_size

        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention
        
        # transpose y to merge all n_heads. B, n_heads, T, head_size -> transpose B, T, n_heads, head_size -> view B, T, Channel_size/n_emb 768 
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # out projection, B, T, C -> B, T, C
        y = self.c_proj(y)
        
        return y
        
        


In [4]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

In [5]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

In [6]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
    
        self.transformer = nn.ModuleDict(
            dict(
                wte = nn.Embedding(config.vocab_size, config.n_embd),
                wpe = nn.Embedding(config.block_size, config.n_embd),
                h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                ln_f = nn.LayerNorm(config.n_embd)
            ))

        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        
        # weight sharing
        self.transformer.wte.weight = self.lm_head.weight

        # weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5

            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            
            if module.bias is not None:
                torch.nn.init.normal_(module.bias, mean=0.0, std=std)
        
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, idx, targets=None):
        B, T = idx.size() # batch , seq_len

        # check if incoming seq_len of idx is within limits
        assert T <= self.config.block_size, f"Cannot proceed as your Sequence len : {T} is more than {self.config.block_size}"

        # forward for token and position encodings
        # shape (T)
        pos = torch.arange(0, T, dtype=torch.int32, device=idx.device)
        pos_emb = self.transformer.wpe(pos) # position embds of shape (T, n_embd)
        token_emb = self.transformer.wte(idx) # token embds of shape (Batch, T/seq_len, n_embd)

        x = pos_emb + token_emb

        # now forward through transformer blocks
        for block in self.transformer.h:
            x = block(x)
        
        # pass through final layernorm
        x = self.transformer.ln_f(x)

        # pass through final LM_HEAD
        logits = self.lm_head(x) # shape (Batch_size, T, vocab_size)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_name):
        """for loading pre-trained GPT model weights from HuggingFace"""
        assert model_name in ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']

        from transformers import GPT2LMHeadModel
        
        print(f"Loading weights from pretrained GPT model: {model_name}")

        # n_layer, n_head, n_embd for each model_name
        config_args = {
            'gpt2':        dict(n_layer=12, n_embd=768, n_head=12), # has 124M params
            'gpt2-medium': dict(n_layer=24, n_embd=1024, n_head=16), # 350M params
            'gpt2-large':  dict(n_layer=36, n_embd=1280, n_head=20), # 774M params
            'gpt2-xl':     dict(n_layer=48, n_embd=1600, n_head=25) # 1558M params
            
        }[model_name]

        config_args['vocab_size'] = 50257 # same for all GPT2 checkpoints
        config_args['block_size']= 1024 # max seq len 1024 for all GPT2 checkpoints

        config = GPTConfig(**config_args)
        model = GPT(config)

        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [key for key in sd_keys if not key.endswith('.attn.bias')] # discard this mask, not a parameter

        # initialize transformer model
        model_hf = GPT2LMHeadModel.from_pretrained(model_name)
        sd_hf = model_hf.state_dict()
        sd_hf_keys = [key for key in sd_hf.keys() if not key.endswith('.attn.masked_bias')] # to discard, not a parameter
        sd_hf_keys = [key for key in sd_hf.keys() if not key.endswith('.attn.bias')] # to discard, not a param

        # transposing these to match openai's Conv1d usage with Linear layer
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        print("=======\nDifference in keys: ", set(sd_keys)- set(sd_hf_keys))
        assert len(sd_keys) == len(sd_hf_keys), f"mismatched keys: sd_keys {len(sd_keys)} != sd_hd_keys {len(sd_hf_keys)}"

        for key in sd_hf_keys:
            if any(key.endswith(w) for w in transposed):
                assert sd_hf[key].shape[::-1] == sd[key].shape
                with torch.no_grad():
                    sd[key].copy_(sd_hf[key].t())
            else:
                # simple copy for other params
                assert sd_hf[key].shape == sd[key].shape
                with torch.no_grad():
                    sd[key].copy_(sd_hf[key])
                    
        return model            



In [7]:
class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B 
        self.T = T 

        with open('input.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'loaded len : {len(self.tokens)}')
        print(f'1 epoch = {len(self.tokens)//(B*T)} batches ')
        self.current_position = 0
    
    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position: self.current_position + (B*T) + 1]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)

        self.current_position += (B*T)
        
        if self.current_position + (B*T+1) > len(self.tokens):
            self.current_position = 0
        return x, y



In [8]:
current_precision = torch.get_float32_matmul_precision()
print(current_precision)

highest


In [9]:
cuda = torch.cuda.is_available()
torch.set_float32_matmul_precision('high')

In [10]:
max_lr = 6e-4
min_lr = 0.1 * max_lr
warmup_steps = 10
max_steps = 5001

def get_lr(iteration):
    if iteration < warmup_steps:
        return max_lr * (iteration + 1) / warmup_steps
    if iteration > max_steps:
        return min_lr
    
    decay_ratio = (iteration - warmup_steps) / (max_steps - warmup_steps)

    assert 0<= decay_ratio <= 1

    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

In [11]:
device = torch.device('cuda')
torch.set_default_device('cuda')

In [12]:
model = GPT(GPTConfig()).to(device=device)

In [13]:
model = torch.compile(model)

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)


In [15]:
train_loader = DataLoaderLite(B=6, T=1024)

loaded len : 338025
1 epoch = 55 batches 


In [16]:
model

OptimizedModule(
  (_orig_mod): GPT(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 768)
      (wpe): Embedding(1024, 768)
      (h): ModuleList(
        (0-11): 12 x Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): CausalSelfAttention(
            (c_attn): Linear(in_features=768, out_features=2304, bias=True)
            (c_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='tanh')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50304, bias=False)
  )
)

In [17]:
sum(p.numel() for p in model.parameters() if p.requires_grad) # 163109376 # 124475904

124475904

In [18]:
device = torch.device('cuda')
torch.set_default_device('cuda')

In [19]:
start= time.time()
for i in range(max_steps):
    t0 = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to(device=device), y.to(device)
    optimizer.zero_grad()

    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        logits, loss = model(x, y)
    
    loss.backward()
    
    # grad clip
    norm = torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

    lr = get_lr(i)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr 

    optimizer.step()
    torch.cuda.synchronize()

    t1 = time.time() 
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_loader.B * train_loader.T) / (dt)
    
    # print happens via CPU, hence wait (synchronize GPU)
    print(f'step : {i+1} | loss: {loss.item()} | dt: {dt:.2f} ms | tokens/sec: {tokens_per_sec:.2f} | norm: {norm:.2f}')
    


end = time.time()

/tmp/ipykernel_6050/3327562475.py:14: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  norm = torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)


step : 1 | loss: 10.967509269714355 | dt: 22181.45 ms | tokens/sec: 0.28 | norm: 27.59
step : 2 | loss: 9.587400436401367 | dt: 160.60 ms | tokens/sec: 38.26 | norm: 9.12
step : 3 | loss: 9.105223655700684 | dt: 162.28 ms | tokens/sec: 37.86 | norm: 3.80
step : 4 | loss: 9.088119506835938 | dt: 159.25 ms | tokens/sec: 38.58 | norm: 7.55
step : 5 | loss: 8.63983154296875 | dt: 160.60 ms | tokens/sec: 38.26 | norm: 2.78
step : 6 | loss: 8.500044822692871 | dt: 159.48 ms | tokens/sec: 38.52 | norm: 3.86
step : 7 | loss: 8.25190258026123 | dt: 159.88 ms | tokens/sec: 38.43 | norm: 2.66
step : 8 | loss: 8.106292724609375 | dt: 158.65 ms | tokens/sec: 38.73 | norm: 2.70
step : 9 | loss: 7.8846235275268555 | dt: 159.03 ms | tokens/sec: 38.63 | norm: 1.78
step : 10 | loss: 7.458106517791748 | dt: 159.05 ms | tokens/sec: 38.63 | norm: 2.76
step : 11 | loss: 7.141818046569824 | dt: 158.42 ms | tokens/sec: 38.78 | norm: 1.80
step : 12 | loss: 6.820505142211914 | dt: 160.68 ms | tokens/sec: 38.24 

In [ ]:
print("final loss: ", loss)
print(f"total time: {end - start} seconds")

final loss:  tensor(0.6168, device='cuda:0', grad_fn=<CompiledFunctionBackward>)
total time: 842.1233501434326 seconds


In [ ]:
torch.save(model.state_dict(), "5k-run.pt")